# Explore grids of CMIP6

This notebook is for exploring the grids used by CMIP6 models. We are mainly interested in the variation in resolution, as we assume all grids will be rectilinear WGS84 grids and will vary mostly in resolution, but are unsure about how much they will vary.

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
from math import radians

In [2]:
cmip6_dir = Path("/beegfs/CMIP6/arctic-cmip6/CMIP6")

Get a list of one file for each model. Each one should have monthly temperature, with a file starting in 2015-01:

In [3]:
fps = list(cmip6_dir.joinpath("ScenarioMIP").glob("*/*/ssp585/*/Amon/tas/*/*/*201501*.nc"))
print("Unique models (files):", len(fps))
fps

Unique models (files): 12


[PosixPath('/beegfs/CMIP6/arctic-cmip6/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ESM4/ssp585/r1i1p1f1/Amon/tas/gr1/v20180701/tas_Amon_GFDL-ESM4_ssp585_r1i1p1f1_gr1_201501-210012.nc'),
 PosixPath('/beegfs/CMIP6/arctic-cmip6/CMIP6/ScenarioMIP/NIMS-KMA/KACE-1-0-G/ssp585/r1i1p1f1/Amon/tas/gr/v20190920/tas_Amon_KACE-1-0-G_ssp585_r1i1p1f1_gr_201501-210012.nc'),
 PosixPath('/beegfs/CMIP6/arctic-cmip6/CMIP6/ScenarioMIP/CNRM-CERFACS/CNRM-CM6-1-HR/ssp585/r1i1p1f2/Amon/tas/gr/v20191202/tas_Amon_CNRM-CM6-1-HR_ssp585_r1i1p1f2_gr_201501-210012.nc'),
 PosixPath('/beegfs/CMIP6/arctic-cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-MM/ssp585/r1i1p1f1/Amon/tas/gn/v20191108/tas_Amon_NorESM2-MM_ssp585_r1i1p1f1_gn_201501-202012.nc'),
 PosixPath('/beegfs/CMIP6/arctic-cmip6/CMIP6/ScenarioMIP/AS-RCEC/TaiESM1/ssp585/r1i1p1f1/Amon/tas/gn/v20200901/tas_Amon_TaiESM1_ssp585_r1i1p1f1_gn_201501-210012.nc'),
 PosixPath('/beegfs/CMIP6/arctic-cmip6/CMIP6/ScenarioMIP/MOHC/HadGEM3-GC31-MM/ssp585/r1i1p1f3/Amon/tas/gn/v20200515/tas_Amon_HadGEM

In [4]:
def get_res(fp):
    with xr.open_dataset(fp) as ds:
        lat_res, lon_res = np.diff(ds.lat)[0].round(2), np.diff(ds.lon)[0].round(2)
        
    return lat_res, lon_res

In [5]:
rows = []
for fp in fps:
    model = fp.parent.parent.parent.parent.parent.parent.parent.name
    inst = fp.parent.parent.parent.parent.parent.parent.parent.parent.name
    lat_res, lon_res = get_res(fp)
    ysize_km = np.round(lat_res * 110)
    xsize_km = np.abs(np.round(lon_res * (111.320 * np.cos(radians(65)))))
    rows.append({
        "inst_model": f"{inst}_{model}",
        "lat_res": lat_res,
        "lon_res": lon_res,
        "ysize_km": ysize_km,
        "xsize_km": xsize_km
    })

df = pd.DataFrame(rows)

/home/kmredilla/miniconda3/envs/cmip6-utils/lib/python3.9/site-packages/xarray/conventions.py:521: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [6]:
df

,inst_model,lat_res,lon_res,ysize_km,xsize_km
0,NOAA-GFDL_GFDL-ESM4,1.00,1.25,110.0,59.0
1,NIMS-KMA_KACE-1-0-G,1.25,1.88,138.0,88.0
2,CNRM-CERFACS_CNRM-CM6-1-HR,0.50,0.50,55.0,24.0
3,NCC_NorESM2-MM,0.94,1.25,103.0,59.0
4,AS-RCEC_TaiESM1,0.94,1.25,103.0,59.0
5,MOHC_HadGEM3-GC31-MM,0.56,0.83,62.0,39.0
6,MOHC_HadGEM3-GC31-LL,1.25,1.88,138.0,88.0
7,MIROC_MIROC6,1.39,1.41,153.0,66.0
8,EC-Earth-Consortium_EC-Earth3-Veg,0.70,0.70,77.0,33.0
9,NCAR_CESM2,0.94,1.25,103.0,59.0


Well it looks like there is one grid which is used more than the others, the grid used by CESM2, CESM2-WACCM, TaiESM1 and NorESM2-MM. The latter three models are based on the CESM2 model, so this makes sense. But this grid also represents a middle ground between the range of resolutions, so this should be a candidate for regridding to.